In [12]:
import torch
import numpy as np
import torchvision

In [6]:
a = [torch.rand(1,4), torch.rand(1, 4)]
print(a)
b = torch.cat(a,0)
b

[tensor([[ 0.2518,  0.0755,  0.3644,  0.9612]]), tensor([[ 0.6993,  0.8270,  0.0911,  0.4161]])]


tensor([[ 0.2518,  0.0755,  0.3644,  0.9612],
        [ 0.6993,  0.8270,  0.0911,  0.4161]])

In [10]:
ret, index = b.sort(0, descending=True)
print(ret)
print(index)
#返回tensor排序和索引排序

tensor([[ 0.6993,  0.8270,  0.3644,  0.9612],
        [ 0.2518,  0.0755,  0.0911,  0.4161]])
tensor([[ 1,  1,  0,  0],
        [ 0,  0,  1,  1]])


In [12]:
index.numel()  #返回元素的总数
# torch.numel()

8

In [4]:
a = torch.rand(3, 224, 224)
a.permute(2,1,0).shape

torch.Size([224, 224, 3])

In [ ]:
torch.nonzero()

In [18]:
a = torch.rand(3,3)
a

tensor([[ 0.3537,  0.6601,  0.8416],
        [ 0.9410,  0.3837,  0.2294],
        [ 0.9501,  0.4583,  0.7400]])

In [10]:
a.nonzero().squeeze()#返回的两列是非零 的点的坐标

tensor([[ 0,  0],
        [ 0,  1],
        [ 0,  2],
        [ 1,  0],
        [ 1,  1],
        [ 1,  2],
        [ 2,  0],
        [ 2,  1],
        [ 2,  2]])

In [20]:
a[a.nonzero()[:,0]]

tensor([[ 0.3537,  0.6601,  0.8416],
        [ 0.3537,  0.6601,  0.8416],
        [ 0.3537,  0.6601,  0.8416],
        [ 0.9410,  0.3837,  0.2294],
        [ 0.9410,  0.3837,  0.2294],
        [ 0.9410,  0.3837,  0.2294],
        [ 0.9501,  0.4583,  0.7400],
        [ 0.9501,  0.4583,  0.7400],
        [ 0.9501,  0.4583,  0.7400]])

In [21]:
a[a.nonzero().squeeze()]  #都是索引出一行，9来自9行，2来自2列，3来自原来的3列

tensor([[[ 0.3537,  0.6601,  0.8416],
         [ 0.3537,  0.6601,  0.8416]],

        [[ 0.3537,  0.6601,  0.8416],
         [ 0.9410,  0.3837,  0.2294]],

        [[ 0.3537,  0.6601,  0.8416],
         [ 0.9501,  0.4583,  0.7400]],

        [[ 0.9410,  0.3837,  0.2294],
         [ 0.3537,  0.6601,  0.8416]],

        [[ 0.9410,  0.3837,  0.2294],
         [ 0.9410,  0.3837,  0.2294]],

        [[ 0.9410,  0.3837,  0.2294],
         [ 0.9501,  0.4583,  0.7400]],

        [[ 0.9501,  0.4583,  0.7400],
         [ 0.3537,  0.6601,  0.8416]],

        [[ 0.9501,  0.4583,  0.7400],
         [ 0.9410,  0.3837,  0.2294]],

        [[ 0.9501,  0.4583,  0.7400],
         [ 0.9501,  0.4583,  0.7400]]])

In [8]:
a[a.nonzero()].shape

torch.Size([9, 2, 3])

In [34]:
a = torch.rand(5)
a.nonzero().shape

torch.Size([5, 1])

In [41]:
def nms(bboxes,scores,threshold=0.5):
    '''
    bboxes(tensor) [N,4] N是所有满足的格子
    scores(tensor) [N,]
    '''
    x1 = bboxes[:,0] #[N, 1] 所有格子的xmin 坐标
    y1 = bboxes[:,1]
    x2 = bboxes[:,2]
    y2 = bboxes[:,3]
    areas = (x2-x1) * (y2-y1) #所有格子面积

    _,order = scores.sort(0,descending=True)
    keep = []
    while order.numel() > 0:
        i = order[0]
        keep.append(i)#i 是最大概率的那个格子

        if order.numel() == 1: #返回tensor总数
            break

        xx1 = x1[order[1:]].clamp(min=x1[i]) #torch.clamp == np.clip
        yy1 = y1[order[1:]].clamp(min=y1[i])
        xx2 = x2[order[1:]].clamp(max=x2[i])
        yy2 = y2[order[1:]].clamp(max=y2[i])

        w = (xx2-xx1).clamp(min=0)
        h = (yy2-yy1).clamp(min=0)
        inter = w*h

        ovr = inter / (areas[i] + areas[order[1:]] - inter) #两个面积相加减去相交的【N, 1]
#         print(ovr.shape)
        ids = (ovr<=threshold).nonzero().squeeze() #把小于阈值的选出来了
#         ids = (ovr<=threshold).nonzero()
        print('ids  ' + str(ids))
#         print(ids.shape)
        if ids.numel() == 0:
            break
        order = order[ids+1]
    return torch.LongTensor(keep)

In [43]:
bboxes = torch.rand(10, 4)
scores = torch.rand(10)
threshold=0.5
ret = nms(bboxes, scores)
print(ret.shape)

ids  tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8])
ids  tensor([ 0,  1,  2,  3,  4,  5,  6,  7])
ids  tensor([ 0,  1,  2,  3,  4,  5,  6])
ids  tensor([ 0,  1,  2,  3,  4,  5])
ids  tensor([ 0,  1,  2,  3,  4])
ids  tensor([ 0,  1,  2,  3])
ids  tensor([ 0,  1,  2])
ids  tensor([ 0,  1])
ids  tensor(0)


D:\annoconda\lib\site-packages\ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app


torch.Size([10])
